# Finding Flat Sinks in fDOM and turbidity

In [1]:
# Imports
import pandas as pd
import math
import copy
import sys
sys.path.insert(1, '../')
from Tools import auxiliary_functions
import Tools.data_movement as dm
import Tools.data_processing as dp

# better print options for checking dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 10)

In [2]:
# load in data
fDOM_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/fDOM_raw_10.1.2011-9.4.2020.csv')
turb_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/turbidity_raw_10.1.2011_9.4.2020.csv')
stage_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/stage_10.1.11-1.1.19.csv')
stage_data = dp.align_stage_to_fDOM(fDOM_data, stage_data)

## fDOM Flat Sinks
Manually found at:
1. idx 85747 -> 86462 (2014-04-16 -> 2014-04-23)
2. 242619 -> 242628 (2018-10-25 -> 2018-10-25)

In [10]:
# put manually found points into a dataframe
cands = [[85747, 85747, 86462]]
cands_df = pd.DataFrame(cands)

cands_df.columns = ["idx_of_peak", "left_base", "right_base"]
cands_df = cands_df.set_index("idx_of_peak")

fdom_df = pd.DataFrame(fDOM_data)
fdom_df.columns = ["timestamp_of_peak", "value_of_peak"]


fdom_df = cands_df.join(fdom_df)

final_fdom_df = fdom_df.reset_index()

del final_fdom_df["left_base"]
del final_fdom_df["right_base"]

final_fdom_df["label_of_peak"] = "FSK"

final_fdom_df = final_fdom_df.reindex(
    columns=["timestamp_of_peak", "value_of_peak", "label_of_peak", "idx_of_peak"]
)

print(final_fdom_df)

csv_path_julian_fdom = "../Data/labeled_data/ground_truths/fDOM/fDOM_FSK/julian_time/fDOM_FSK_0k-300k.csv"
csv_path_datetime_fdom = "../Data/labeled_data/ground_truths/fDOM/fDOM_FSK/datetime/fDOM_FSK_0k-300k.csv"

datetime_df = copy.deepcopy(final_fdom_df)

for i, row in datetime_df.iterrows():
    jul_time = datetime_df.loc[i, "timestamp_of_peak"]

    dt = dp.julian_to_datetime(jul_time)
    dt = dt.isoformat()

    datetime_df.loc[i, "timestamp_of_peak"] = dt

final_fdom_df = final_fdom_df.set_index("timestamp_of_peak")
datetime_df = datetime_df.set_index("timestamp_of_peak")
final_fdom_df.to_csv(csv_path_julian_fdom)
datetime_df.to_csv(csv_path_datetime_fdom)

   timestamp_of_peak  value_of_peak label_of_peak  idx_of_peak
0    2456947.65625     21.32691384        FSK         85747   


## Turbidity Flat Sinks
None manually found